
# S07T01_Hypothesis_Testing
Sprint 7.Task 1. Hypothesis Testing

## Description
Learn how to perform hypothesis testing with Python.

Level 1
- Exercise 1
Take a data set you like and select an attribute from the data set. Calculate the p-value and say whether it disproves the null hypothesis by taking an alpha of 5%. 

Level 2
- Exercise 2
Continue with the data set you like and select two other attributes from the data set. Calculate the p-values and say whether they disprove the null hypothesis by taking an alpha of 5%.

Level 3 
- Exercise 3
Continue with the data set you like and select three attributes from the data set. Calculate the p-value and say whether it disproves the null hypothesis by taking an alpha of 5%.


## Level 1
### - Exercise 1
Take a data set you like and select an attribute from the data set. Calculate the p-value and say whether it disproves the null hypothesis by taking an alpha of 5%. 

Explanation of p-value:  ( 🗨️ from https://en.wikipedia.org/wiki/P-value)
![](2022-02-09-18-38-42.png)

We are going to select the dataset of the 2022 Hackathon MWC Barcelona (Mobile World Congress)

## Dataset Information:
## 🖥 RETO
### Background  
Nuwefruit es una startup que busca revolucionar los hábitos de la población fomentando el cosumo de fruta a diario. Por este motivo, la empresa está especializada en la venta de fruta a domicilio, que gracias a su algoritmo de optimización de la última milla le permite tener unos costes logisticos muy bajos. Esto permite que Nuwefruit pueda vender fruta a un precio inferior al de su competencia. Su catálogo se basa en la venta de más de 20 tipos de frutas, que son las que presentan las mejores propiedades nutritivas.

### Overview: the dataset and challenge

Se emplearan dos datasets: el primero contiene datos de los clientes de Nuwefruit y el otro contiene los datos de los pedidos realizados por estos.

El dataset de clientes 'CLIENT TABLE' contiene las siguientes variables:


|Columna  |Descripción  |
|---------|---------|
|CLIENT ID:| Identificador único del cliente  
|CLIENT_SEGMENT:| Segmento de clientes  
|AVG CONSO: | Consumo medio mensual del cliente calculado a finales de 2020 (en piezas de fruta)
|AVG BASKET SIZE: |Tamaño medio de la cesta del cliente calculado a finales de 2020 (en piezas de fruta)
|RECEIVED_COMMUNICATION:| 1 = Recibió promoción de sus productos / 0 = no la recibió

El dataset de clientes 'ORDERS TABLE' contiene las siguientes variables:

|Columna  |Descripción  |
|---------|---------|
|CLIENT ID:| Identificador único del cliente
NB PRODS:| Número de 'prods' de la variedad de fruta en el pedido (1 prod = 10 piezas de fruta)
ORDER ID:| Identificador único del pedido
FRUIT_PRODUCT:| Variedad de fruta


### Objetivos
Haz un analisis exploratorio de los datos que permita:
- Analizar las ventas y la actividad de los clientes
- Evaluar el impacto de la promoción
- Realiza un modelo predictivo que permita conocer el tipo de segmento al que pertenece cada cliente en función de las siguientes variables predictoras: Descargar test_x. (Se ha de predecir la variable (CLIENT_SEGMENT)).
### Evaluación
- 900 puntos -> Objetivos. Los objetivos se evaluarán en función del EDA realizado y del f1-score(macro) obtenido al comparar las predicciones de tu algoritmo cuando se les mete como input las variables del dataset 'test_x'.
- 200 puntos -> Cálidad del código
- 100 puntos -> Documentación

### Entrega
Has de pegar el link a tu repositorio.

Este ha de tener:

- El código con el que has realizado el EDA y el modelo predictivo.

- Un archivo 'results.csv' con las predicciones de tu algoritmo al meterle como input el dataset test_x. Este archivo con los resultados ha de tener una columna con el 'CLIENT ID' y otra con las predicciones del 'CLIENT_SEGMENT'. Tal y como en el siguiente ejemplo: Ejemplo results.csv



In [49]:
# Import libraries
import numpy as np
import pandas as pd

In [50]:
# Importar los datasets
df_clients = pd.read_csv("mwc22-client_table.csv", decimal = ',')
df_orders = pd.read_csv("mwc22-orders_table.csv")
df_clients_test_x = pd.read_csv("mwc22-client_table+-+test_x.csv", decimal = ',')

In [51]:
# Limpieza de nombres de columnas df_clients
df_clients.columns = df_clients.columns.str.lower()
df_clients.columns = df_clients.columns.str.replace(" ","_")
df_clients.rename_axis('client_id', inplace=True)
df_clients.columns

Index(['client_id', 'client_segment', 'avg_conso', 'avg_basket_size',
       'received_communication'],
      dtype='object')

In [52]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35884 entries, 0 to 35883
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   client_id               35884 non-null  int64  
 1   client_segment          35884 non-null  int64  
 2   avg_conso               35884 non-null  float64
 3   avg_basket_size         35884 non-null  float64
 4   received_communication  35884 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 1.4 MB


In [53]:
df_clients.sort_index(inplace=True)

In [54]:
# Limpieza de nombres de columnas df_orders
df_orders.columns = df_orders.columns.str.lower()
df_orders.columns = df_orders.columns.str.replace(" ","_")
df_orders.rename_axis('client_id', inplace=True)
df_orders.columns

Index(['client_id', 'nb_prods', 'order_id', 'fruit_product'], dtype='object')

In [55]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66912 entries, 0 to 66911
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   client_id      66912 non-null  int64 
 1   nb_prods       66912 non-null  int64 
 2   order_id       66912 non-null  int64 
 3   fruit_product  66912 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.0+ MB


In [56]:
round(df_clients.describe(include='all'),2)

,client_id,client_segment,avg_conso,avg_basket_size,received_communication
count,35884.00,35884.00,35884.00,35884.00,35884.00
mean,27060578.92,3.12,64.53,181.22,0.51
std,8835076.38,1.51,64.38,129.60,0.50
min,18073108.00,1.00,0.83,10.00,0.00
25%,20533114.25,2.00,20.83,100.00,0.00
50%,24621903.00,3.00,50.00,160.00,1.00
75%,32985383.25,4.00,88.17,225.00,1.00
max,48365940.00,6.00,2433.33,3400.20,1.00


In [57]:
round(df_orders.describe(),2)

,client_id,nb_prods,order_id
count,66912.00,66912.00,6.691200e+04
mean,26134065.11,4.53,6.722533e+08
std,8473596.01,5.79,3.205826e+06
min,18070505.00,-80.00,6.638335e+08
25%,20174269.00,1.00,6.694803e+08
50%,24380549.00,3.00,6.719971e+08
75%,25387075.00,5.00,6.750893e+08
max,48365861.00,198.00,6.832132e+08


In [58]:
round(df_orders.describe(include='all'),2)

,client_id,nb_prods,order_id,fruit_product
count,66912.00,66912.00,6.691200e+04,66912
unique,NaN,NaN,NaN,24
top,NaN,NaN,NaN,Kiwi
freq,NaN,NaN,NaN,6431
mean,26134065.11,4.53,6.722533e+08,NaN
std,8473596.01,5.79,3.205826e+06,NaN
min,18070505.00,-80.00,6.638335e+08,NaN
25%,20174269.00,1.00,6.694803e+08,NaN
50%,24380549.00,3.00,6.719971e+08,NaN
75%,25387075.00,5.00,6.750893e+08,NaN


## Level 2
### - Exercise 2
Continue with the data set you like and select two other attributes from the data set. Calculate the p-values and say whether they disprove the null hypothesis by taking an alpha of 5%.

## Level 3 
### - Exercise 3
Continue with the data set you like and select three attributes from the data set. Calculate the p-value and say whether it disproves the null hypothesis by taking an alpha of 5%.
